# 🏠 Multimodal ML: Housing Price Prediction Using Images + Tabular Data
Predict housing prices using both structured features and images with a combined model.

In [ ]:
# ✅ Step 0: Install Required Libraries
!pip install pandas numpy scikit-learn torch torchvision matplotlib --quiet

In [ ]:
# 📥 Step 1: Load Data (Sample Simulated)
import pandas as pd
import numpy as np
import os
from PIL import Image
from torchvision import transforms

# Simulated CSV (Replace with your real dataset)
# CSV should contain a 'filename' column for image file name and 'price' as target
csv_path = 'housing_data.csv'  # Replace with actual path
image_folder = 'images/'       # Replace with your folder
df = pd.read_csv(csv_path)
df = df.dropna()

# Example of loading and transforming images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def load_image(img_path):
    img = Image.open(os.path.join(image_folder, img_path)).convert('RGB')
    return transform(img)

# Load all images
df['image_tensor'] = df['filename'].apply(load_image)

In [ ]:
# 🔍 Step 2: Tabular Feature Normalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Drop unused and label columns
tabular_features = df.drop(columns=['filename', 'price', 'image_tensor'])
scaler = StandardScaler()
X_tab = scaler.fit_transform(tabular_features)

y = df['price'].values
X_img = torch.stack(df['image_tensor'].tolist())

# Train/test split
X_tab_train, X_tab_test, X_img_train, X_img_test, y_train, y_test = train_test_split(
    X_tab, X_img, y, test_size=0.2, random_state=42
)

In [ ]:
# 🧠 Step 3: Define Multimodal Neural Network
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiModalNet(nn.Module):
    def __init__(self, tabular_dim):
        super(MultiModalNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.tabular = nn.Sequential(
            nn.Linear(tabular_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.fc = nn.Linear(32 + 64, 1)

    def forward(self, x_img, x_tab):
        x_img = self.cnn(x_img)
        x_img = x_img.view(x_img.size(0), -1)
        x_tab = self.tabular(x_tab)
        x = torch.cat((x_img, x_tab), dim=1)
        return self.fc(x).squeeze(1)

In [ ]:
# 🚆 Step 4: Train the Model
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiModalNet(X_tab_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_ds = TensorDataset(torch.tensor(X_img_train).float(), torch.tensor(X_tab_train).float(), torch.tensor(y_train).float())
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

# Training loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for xb_img, xb_tab, yb in train_dl:
        xb_img, xb_tab, yb = xb_img.to(device), xb_tab.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb_img, xb_tab)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

In [ ]:
# 📊 Step 5: Evaluate Model
model.eval()
with torch.no_grad():
    preds = model(X_img_test.to(device).float(), torch.tensor(X_tab_test).float().to(device))
    preds = preds.cpu().numpy()

from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print("MAE:", mae)
print("RMSE:", rmse)